In [7]:
import requests
import lxml.html
from bs4 import BeautifulSoup
import urllib.request
import time
import pickle
import pandas as pd
from datetime import datetime

In [2]:
url0 = 'http://www.fudousan.or.jp/system/?act=l&type=12&pref=13'

In [3]:
page = requests.get(url0)
soup=BeautifulSoup(page.text,"lxml")

In [4]:
bodys=[]
bodys.append(soup.body)

In [5]:
#検索件数を取得し、ページ数を計算する
for i in bodys[0].find_all('th'):
    if (i.text).find('検索結果')!=-1:
        end=(i.text).find('件')
        num=int(i.text[6:end])

page_num=num//20+1

In [6]:
#検索全ページのurlを取得
urls=[]
for i in range(2,page_num):
    urls.append('http://www.fudousan.or.jp/system/?act=l&type=12&pref=13&n=20&p={}&v=off,s='.format(i))

In [16]:
#全ページのhtmlを5秒おきに取得
pages=[]
# for i in range(len(urls)):
for i in range(3):
    print(i)
    time.sleep(5)
    page_i=requests.get(urls[i])
    pages.append(page_i)

0
1
2


In [17]:
#検索を実行した時刻を取得
searched_date=datetime.now().strftime("%Y%m%d")
print(searched_date)

20180716


In [18]:
# 取得したページのhtmlをpickle化
with open('storage/pages_{}.pickle'.format(searched_date),'wb') as f:
    pickle.dump(pages, f)

In [19]:
# htmlをpickle化したものをロード
load_date='20180716'
with open('storage/pages_{}.pickle'.format(load_date),'rb') as f:
    pages=pickle.load(f)

In [20]:
#htmlをパースして、bodyを取り出す
for i in pages:
    soup_i=BeautifulSoup(i.text,"lxml")
    bodys.append(soup_i)

In [21]:
#ページ内のbit番号を取得
bids=[]
for bd in bodys:
    for i,tag in enumerate(bd.find_all('a')):
        if (tag.get('href')!=None):
            bid_st=(tag.get('href')).find('bid=')
            bid_len=(tag.get('href'))[bid_st:].find('&')
            if bid_st != -1:
                bid_num=tag.get('href')[bid_st+4:bid_st+bid_len]
                bids.append(bid_num)

In [22]:
#重複したbit番号を削除
bids=list(set(bids))

In [23]:
#各物件のlinkをリストに格納
links=[]
for i in bids:
    links.append('http://www.fudousan.or.jp/system/?act=d&type=12&pref=13&n=20&p=1&v=on&s=&bid={}'.format(i))

### 各物件画面

In [24]:
#各物件のhtmlを五秒おきに取得
link_pages=[]
for i in range(len(links)):
    print(i)
    time.sleep(5)
    link_page_i=requests.get(links[i])
    link_pages.append(link_page_i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [25]:
# 取得した物件ページのhtmlをpickle化
with open('storage/link_pages_{}.pickle'.format(searched_date),'wb') as f:
    pickle.dump(link_pages, f)

In [26]:
# htmlをpickle化したものをロード
with open('storage/link_pages_{}.pickle'.format(load_date),'rb') as f:
    link_pages=pickle.load(f)

In [27]:
#htmlをパースして、bodyを取り出す
link_bodys=[]
cnt=0
for i in link_pages:
    cnt+=1
    print(cnt)
    link_soup_i=BeautifulSoup(i.text,"lxml")
    link_bodys.append(link_soup_i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [28]:
#物件情報の取得
specs=[]

for i,link_bd in enumerate(link_bodys):
    print(i)
    keys=[]
    values=[]
    
    for s in link_bd.find_all('span'):
        if (s.text).find('価格') != -1:
            price=s.text.split()[1][:-2]
            keys.append('price')
            values.append(price)
        
    for t in link_bd.find_all('tr'):
        if (t.find('th')!=None) & (t.find('td')!=None):
            th_tags=t.find_all('th')
            td_tags=t.find_all('td')
            for i in range(len(th_tags)):
                keys.append(th_tags[i].text)
                values.append(td_tags[i].text)
    spec=dict(zip(keys, values))
    specs.append(spec)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [29]:
# 取得した物件ページのspecをpickle化
# with open('storage/specs_{}.pickle'.format(searched_date),'wb') as f:
#     pickle.dump(specs, f)

In [30]:
#dictionaryに保存したデータをcsvに出力する
df_specs=pd.DataFrame(specs)
col=['price',
        '詳細情報\xa0',
        '所在地',
        '交通',
        '㎡／坪単価',
        '専有面積',
        '間取り',
        '間取り内訳',
        '物件種目',
        '築年月',
        '構造',
        '階数',
        '向き',
        'バルコニー等面積',
        '駐車場',
        '設備',
        '総戸数',
        '販売戸数',
        '敷地面積',
        '敷地の権利形態',
        '借地料および借地期間',
        '敷金',
        '保証金',
        '管理形態',
        '管理会社',
        '管理費等',
        '修繕積立金',
        '修繕積立基金',
        '施設費用/償却費',
        '用途地域',
        'その他費用',
        '施工会社',
        '現況',
        '引き渡し時期',
        '国土法届出',
        '工事完了年月',
        '備考１',
        '備考2',
        '特記',
        '取引態様',
        '情報登録日',
        '次回更新予定日',
        '物件HP',
        '不動産会社詳細']

output=pd.DataFrame(columns=col)
for i,c in enumerate(col):
    output[output.columns[i]]=df_specs[c]
output.to_csv('intermediate_data/spec_{}.csv'.format(searched_date))

In [31]:
output.shape

(80, 44)